[![](./media/Db2_header_3.png)](https://www.ibm.com/demos/collection/db2-database/)
<a id="top">

In [35]:
%run refresh.ipynb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Installing Db2 on OpenShift

This lab will take the user through the steps required to get Db2 running on a OpenShift cluster. In order to simulate a cluster we use CentOS 7.7 and OpenShift on a virtual machine. See the below blog for details on how to deploy the base environment yourself.

https://www.ibm.com/cloud/blog/how-to-build-a-db2-for-openshift-environment-for-your-desktop

### How to Copy Code and Examples
Throughout this lab there are code samples that need to be copied and modified in a text editor. Any commands that need to be executed from a command line are found in grey boxes (an example is found below) has been designed to be easily copied.

In [36]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
""<pre id=112 onmousedown="window.clipline(112)" onmouseup="window.reset(112)  ">
Sample commands are found in cells like this.
</pre>
</div>


The entire contents of the text in the cell will be automatically copied when you click on the cell. The color of the background will change color briefly to indicate that the copy has completed. To paste commands into a terminal window, use the key combination **Control-Shift-v**. 


### Background to our Deployment Environment

The Deployment environment is a CentOS 7.7 Operating system with Openshift 3.11 Single node Install and Helm 2.16 deployed with an installation of Tiller.

If you are interested in deploying such an environment yourself for your own use you can find the instructions at: 
    

In [37]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=1 onmousedown="window.clipline(1)" onmouseup="window.reset(1)">
https://www.ibm.com/cloud/blog/how-to-build-a-db2-for-openshift-environment-for-your-desktop

## Use the Slider to browse through the background on DB2u and Openshift

In [38]:
import ipywidgets as wg
from IPython.display import Image
from ipywidgets import HBox
def f(Slides):
        return Image(filename='./media/db2u/Slide'+str(Slides)+'.png', width=1400)

wg.interact(f, Slides=wg.IntSlider(min=1,max=31,step=1,layout=dict(width='80%',height='80px'),orientation='horizontal'))
display()

interactive(children=(IntSlider(value=1, description='Slides', layout=Layout(height='80px', width='80%'), max=…

## Passwords for this lab
### Operating system login
        UserID: db2pot
        password: 123qwe123

### Openshift Login 
        UserID: admin
        password: redhat

### Setup Openshift Environment for Db2

The next steps will be to ensure the container groups and Selinux permissions are set so that our Db2 containers can install and operate correctly.

Open up a command line as per instrctions below and then use the single click to copy the following commands and then past them into the command line window. All instructions in black cells are to be executed on the command line.

### Getting Started
To run a new command terminal Click on the button below.

In [39]:
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Command Line",button_style='primary')
output = widgets.Output()
display(button, output)
def on_button_clicked(b):
        ! gnome-terminal --window-with-profile=db2pot --working-directory=/home/db2pot
button.on_click(on_button_clicked)

Button(button_style='primary', description='Command Line', style=ButtonStyle())

Output()

It may be easier to keep a terminal window on top of the Jupyter notebook when running these commands. When you have a terminal window displayed, **right click on the title bar** and select **Always on Top** to keep the screen visible during the duration of the lab.

Now in the Command Terminal we will apply the Selinux settings to enable containers to manage data on the file system.

In [40]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=2 onmousedown="window.clipline(2)" onmouseup="window.reset(2)">
sudo setsebool -P container_manage_cgroup true

Create the directory in which the Db2 service will create its persistent data.

In [41]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=3 onmousedown="window.clipline(3)" onmouseup=" window.reset(3)">
mkdir -p /data/db2vol1

Now we will provide the sufficient permissions for the service via containers to own the directories.

In [42]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=4 onmousedown="window.clipline(4)" onmouseup="window.reset(4)">
chmod 777 -R /data/db2vol1

In [43]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=5 onmousedown="window.clipline(5)" onmouseup="window.reset(5)">
sudo chgrp root -R /data/db2vol1

Enable the directory to be managed and accessed by our OpenShift environment.

In [44]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=7 onmousedown="window.clipline(7)" onmouseup="window.reset(7)">
sudo semanage fcontext -a -t container_file_t "/data/db2vol1(/.*)?"

In [45]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=6.1 onmousedown="window.clipline(6.1)" onmouseup="  window.reset(6.1)">
sudo restorecon -Rv /data/db2vol1

### Building our Db2 Openshift Project

First Step is to create an new project called Db2. This will create the OpenShift project and the Kubernetes namespace.

To login, use the command below with user **admin** and default namespace using password **redhat** and complete the following steps.

In [46]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=8 onmousedown="window.clipline(8)" onmouseup="  window.reset(8)">
oc login -u admin -n default

In [47]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=9 onmousedown="window.clipline(9)" onmouseup="window.reset(9)">
oc new-project db2

Now we will give Helm's Tiller service permission to create the Db2 installation in our Openshift Db2 project.

In [48]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=10 onmousedown="window.clipline(10)" onmouseup="  window.reset(10)">
oc policy add-role-to-user admin "system:serviceaccount:${TILLER_NAMESPACE}:tiller"

Now we are going to clone the Db2 Github project that installs and configures Db2 for OpenShift. 
The branch of the project we will use has been specifically congured to run in a single OpenShift Node deployment. If you were going to deploy this on a multi-node cluster you would omit the branch parameter.

In [49]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=11 onmousedown="window.clipline(11)" onmouseup=" window.reset(11)">
cd /home/db2pot

In [50]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=12 onmousedown="window.clipline(12)" onmouseup=" window.reset(12)">
git clone https://github.com/IBM/charts

### Apply Db2 Security Permissions and Roles to our Project

Now we will apply the Db2 Roles, Security Context and Bindings required in the Db2 project for the Db2 service to run.

In [51]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=13 onmousedown="window.clipline(13)" onmouseup=" window.reset(13)">
cd /home/db2pot/charts/stable/ibm-db2/ibm_cloud_pak/pak_extensions

In [44]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=131 onmousedown="window.clipline(131)" onmouseup=" window.reset(131)">
 ./pre-install/clusterAdministration/createSecurityClusterPrereqs.sh

In [43]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=133 onmousedown="window.clipline(133)" onmouseup=" window.reset(133)">
 ./pre-install/namespaceAdministration/createSecurityNamespacePrereqs.sh db2

In [ ]:
RELEASE_NAME="db2u"
$ PASSWORD="redhat"
$ oc create secret generic db2u-db2u-ldap-bluadmin --from-literal=password="redhat"

In [36]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=132 onmousedown="window.clipline(132)" onmouseup=" window.reset(132)">
oc create secret generic db2u-db2u-instance --from-literal=password="redhat"

In [36]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=232 onmousedown="window.clipline(232)" onmouseup=" window.reset(232)">
oc create secret generic db2u2-db2u-ldap-bluadmin --from-literal=password="redhat"


## Obtaining an API Key to Access our Images
before Openshift can access our DB2 Community Edition Image we need to obtain an API access code from the IBm Cloud to provide Openshift so it can pull the required images. To do so we need to follow the below process:
    * Log into IBM Cloud https://cloud.ibm.com/login
    * Select Manage > Access(IAM) from the toolbar
    * Select API keys from the side menu, click Create an IBM Cloud API Key
    * Enter a Name and Description. Click create
    * You can copy the API key or select to download the key and save it.
    * Create image registry secret in your current project


In [52]:
import ipywidgets as wg
from IPython.display import Image
def f(Slides):
    return Image(filename='./media/iamkey/Slide'+str(Slides)+'.png',width=1400)

wg.interact(f, Slides=wg.IntSlider(min=1,max=7,step=1,layout=dict(width='80%')))
display()

interactive(children=(IntSlider(value=1, description='Slides', layout=Layout(width='80%'), max=7, min=1), Outp…

# Creating the API Key Secret

To create the API secret, take the API key you created and add to the below command and hit **Shift-Enter** keys together or choose the run button at the top of your screen to execute the code.

In [37]:
%%system 

oc delete secret ibm-registry 

['Error from server (NotFound): secrets "ibm-registry" not found']

In [38]:
%%system 
 oc create secret docker-registry ibm-registry \
    --docker-server=icr.io \
    --docker-username=iamapikey \
    --docker-password="ENTER API KEY HERE"


['secret/ibm-registry created']

In [88]:
%%system 
    oc secrets link db2u ibm-registry --for=pull

[]

### Create the Db2 Persistent Volume and Volume Claim

First we will use the **vi** editor to edit and create our volume and claim definition.
#### We have actually created the db2vol.yaml file for you so unless you wish to change or work wiht this your self you can skip to the apply of the Persistent Volume. 
Otherwise feel free to work with the script !

In [53]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=15 onmousedown="window.clipline(15)" onmouseup="window.reset(15)">
vi /home/db2pot/notebooks/db2vol.yaml

Now you can apply the db2vol.yaml file and create the volumes for db2 to install into. 

In [54]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
""<pre id=17 onmousedown="window.clipline(17)" onmouseup=" window.reset(17)">
oc apply -f /home/db2pot/notebooks/db2vol.yaml 

## Now Lets Install Db2!

First we are going to move to the db2 deployment directory then run the Db2 install with the required parameters for a small install.

In [55]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=181 onmousedown="window.clipline(181)" onmouseup=" window.reset(181)">
cd /home/db2pot/charts/stable/ibm-db2/ibm_cloud_pak/pak_extensions/common


In [56]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=19 onmousedown="window.clipline(19)" onmouseup="window.reset(19)">
./db2u-install --db-type db2oltp --namespace db2 --release-name db2u --existing-pvc db2pvc --tiller-namespace tiller --cpu-size 1 --memory-size 3Gi

The above installs a Db2 database with configurations for OLTP, in our Namespace / Project Db2, using our defined Db2 Persistent volume claim and sets its cpu limit to 1 cores and a memory allocation of 3gb.

## Now we are going to monitor the deployments progress

First of all we are going to get the status of the pods. You will have to run this command multiple times as it will take a few minutes for the install to download the containers and install them in your project.

In [57]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=20 onmousedown="window.clipline(20)" onmouseup="window.reset(20)">
oc get pods

When your installation is complete you should see the following status with the db2u-db2u-0 pod status at Running and the ready status as 1/1.

```table
NAME                                READY       STATUS  RESTARTS    AGE
db2u-db2u-0                         1/1         Running     0       10m
db2u-db2u-engn-update-job-cfxp2     0/1         Completed   0       10m  
db2u-db2u-ldap-74b767d76-sbct6      1/1         Running     0       10m
db2u-db2u-nodes-cfg-job-7m7bz       0/1         Completed   0       10m
db2u-db2u-restore-morph-job-bnxfx   0/1         Completed   0       10m
db2u-db2u-sqllib-shared-job-bjrwc   0/1         Completed   0       10m
db2u-db2u-tools-c47c5b565-svwcw     1/1         Running     0       10m
db2u-etcd-0                         1/1         Running     0       10m
db2u-etcd-1                         1/1         Running     0       10m
db2u-etcd-2                         1/1         Running     0       10m
```

If you want to just get incremental updates of changes to the pods status you can try running "oc get pods -w". This command will give you the current status at the time of running and any incremental changes in status.

The last pod status change you will see is the `db2u-db2u-restore-morph` pod/job status will be `Completed`. Ctrl-C to quit out at that point.

## Finding our what address to connect to Db2 on

To find out the ports that we can connect to Db2, first run the following command.

In [58]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=21 onmousedown="window.clipline(21)" onmouseup="  window.reset(21)">
oc get svc

```
NAME                 TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                                    AGE
db2u-db2u            ClusterIP   172.30.96.174    <none>        50000/TCP,50001/TCP,25000/TCP,25001/TCP,…  2m
db2u-db2u-engn-svc   NodePort    172.30.169.115   <none>        50000:31020/TCP,50001:31578/TCP            2m
db2u-db2u-internal   ClusterIP   None             <none>        50000/TCP,9443/TCP                         2m
db2u-db2u-ldap       NodePort    172.30.122.233   <none>        389:30154/TCP                              2m
db2u-etcd            ClusterIP   None             <none>        2380/TCP,2379/TCP                          2m
```

Db2 exposes its service externally via the `db2u-db2u-engn-svc` service. In this case, `31020` is the non-SSL port Db2 will be listening on, and `31578` is the port that will support SSL connectivity. They are mapped to the `db2u-db2u-0` `50000/50001` ports that Db2 operates on.

To get your IP address for your server for external communication, use the following (if you are running on VMware):

In [59]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=22 onmousedown="window.clipline(22)" onmouseup=" window.reset(22)">
ip addr show | grep 'scope global noprefixroute'

A sample output is shown below. The `inet` value is what you would use for connecting to Db2 along with the appropriate port number.

```python
result: inet 192.168.154.132/24 brd 192.168.154.255 scope global noprefixroute dynamic ens33
```

## Lets Connect!

The following commands are run using the Jupyter notebook. Select each cell and hit **Shift-Enter** keys together or chose the run button at the top of your screen to execute the code. Alternatively you can place your cursor on the grey sell and select the Run button at the top of the screen.

In [35]:
%run db2.ipynb

Db2 Extensions Loaded.


**Note:** The following command assumes an IP address of `192.168.154.132` and port `30920`. Your image may have different values so make changes as necessary.

**If you are running this lab on skytap, your Host-ip is most likely 10.0.0.1**

In [38]:
%sql connect to bludb user db2inst1 using redhat host <HOST IP>  port <PORT>

Connection successful.


If the connection is not successful, it may be due to Db2 not having started completely. You may need to wait for a few minutes for Db2 to start accepting connections. Once you do have a connection, you need to add some sample data to the database. The BLUDB database does not have the sample tables in it, so issue the following command to populate the EMPLOYEE and DEPARTMENT tables.

In [37]:
%sql -sampledata

Sample tables [EMPLOYEE, DEPARTMENT] created.


At this point you can try various SQL commands since we have established a connection to Db2.

In [41]:
%sql -grid SELECT * FROM EMPLOYEE

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': False, 'defa…

In [ ]:
%sql -bar SELECT WORKDEPT, AVG(SALARY) FROM EMPLOYEE GROUP BY WORKDEPT

## Monitoring Activities
You can get an overview of the activities within the cluster by using the Openshift dashboard.

Once in the console, Navigate to the Db2 project, click on applications, then select pods to view pod status and browse from there.

Click on here and login as **user:admin / pwd:redhat **
            https://localhost:8443/login

[![](./media/Openshift.png)](https://localhost:8443/login)
<a id="top">

All the commands we run from the `oc` environment to describe, browse logs and so on are available via the console.

## Simulating a Db2 Crash
You can simulate a crash in openShift by deleting the Db2 pod. When you kill the pod, OpenShift will restart the pod automatically for you. The first command will delete the Db2 pod, while the next command will watch the systems as the pod is dropped and recreated. To watch this process work, issue the first command in one command window and the second in another. 

In [60]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=23 onmousedown="window.clipline(23)" onmouseup="window.reset(23)">
oc delete pod db2u-db2u-0

Run the following command in the second window to watch the recovery occur.

In [61]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=23.1 onmousedown="window.clipline(23.1)" onmouseup="window.reset(23.1)">
oc get pods -w

Keep this status window so you can watch the delete step in the next section.

## Delete Db2 Deployment
We can delete the Db2 pods by using Helm. This will properly remove Db2 since you can see that just deleting the pod will not work!

In [62]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=24 onmousedown="window.clipline(24)" onmouseup="window.reset(24)">
helm delete db2u --purge

Check on the status of the purge.

In [63]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=24.1 onmousedown="window.clipline(24.1)" onmouseup="window.reset(24.1)">
oc get pods

## Restart Db2 Install (It's much faster!)
Since we still have the Db2 images in the local Docker repository, and the persistent volume is still available, the recreation of Db2 pods is extremely quick.

In [64]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=25 onmousedown="window.clipline(25)" onmouseup=" window.reset(25)">
cd /home/db2pot/Db2/deployment

In [65]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=26 onmousedown="window.clipline(26)" onmouseup="  window.reset(26)">
./db2u-install --db-type db2oltp --namespace db2 --release-name db2u --existing-pvc db2pvc --tiller-namespace tiller --cpu-size 1 --memory-size 3Gi

Check its progress to see how long this deployment takes.

In [66]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=27 onmousedown="window.clipline(27)" onmouseup=" window.reset(27)">
oc get pods

To get an even more detailed view of the Db2 install, we can describe the Db2 pod.

In [67]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=28 onmousedown="window.clipline(28)" onmouseup=" window.reset(28)">
oc describe pod db2u-db2u-0 

Or we can view its installation log.

In [68]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=29 onmousedown="window.clipline(29)" onmouseup=" window.reset(29)">
oc logs db2u-db2u-0

### We hope you have enjoyed this lab, to view more labs in this collection click on the footer bar below!

[![](./media/db2_dte_footer.png)](https://www.ibm.com/demos/collection/db2-database/)
<a id="top">

#### Credits: IBM 2020, Phil Downey [phil.downey1@ibm.com], George Baklarz [baklarz@ca.ibm.com]